In [ ]:
!wget -q https://dlcdn.apache.org/spark/spark-3.3.1/spark-3.3.1-bin-hadoop3.tgz
!tar xf spark-3.3.1-bin-hadoop3.tgz
!pip install -q findspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.3.1-bin-hadoop3"

import findspark
findspark.init()

from pyspark import SparkContext
sc = SparkContext("local","first app")

데이터

points1.csv: 10개의 점ID와 2차원 (xy좌표) 점의 집합

points2.csv: 100,000개의 2차원 (xy좌표) 점의 집합

문제

points1의 각 점 X에 대해 다른 points1의 점들 중 X를 가장 가까운 점으로 하는 

points2의 점의 개수를 구하시오.

즉, 점 X의 ReverseClosestPoint(X)를 {u∈points2|∀v∈points1,d(u,X)≤d(u,v)}라는 

집합으로 정의하였을 때 각 점 X에 대해 ReverseClosestPoint(X)의 크기를 출력하는 

것이 문제임.

출력: 점ID, |ReverseClosestPoint(X)|

HINT: points1은 broadcasting하고 ReverseClosestPoint(X)의 크기는 accumulator로 

카운트 하시오.

In [ ]:
points1 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/points1.csv",1)
points2 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/points2.csv",1)


In [ ]:
points1 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/points1.csv",1)
points2 = sc.textFile("/content/drive/MyDrive/Colab Notebooks/데이터분석을위한프로그래밍/points2.csv",1)

point1_bc =sc.broadcast(points1.map(lambda x:eval(x)).collect())
point1_bc.value

In [ ]:
point2 =points2.map(lambda x:eval(x))
point2.take(5)

In [ ]:
def dist(a,b):
  return ((a[0]-b[0])**2 + (a[1]-b[1])**2)**0.5
def distance_list(x):
  distanceL =[]
  for i in range(len(point1_bc.value)):
    distanceL.append(dist(x,point1_bc.value[i][1:]))
  return distanceL

point2.map(lambda x: distance_list(x)).map(lambda x : np.argmin(x)).map(lambda x :(x,1)).reduceByKey(lambda x,y: x+y).collect()

[(1, 8319),
 (4, 15936),
 (6, 14729),
 (2, 15088),
 (7, 8222),
 (3, 10818),
 (8, 1632),
 (5, 6201),
 (9, 4319),
 (0, 14736)]